## Preprocessing

In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.3'
spark_version = 'spark-3.4.3'
os.environ['SPARK_VERSION']=spark_version

In [ ]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 233 kB in 2s (124 kB/s)
Reading package lists... Done


In [ ]:
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

In [ ]:
# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Charity Data").getOrCreate()

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df.drop(columns=['EIN', 'NAME'], inplace=True)


In [ ]:
# Determine the number of unique values in each column.
unique_value_counts = application_df.nunique()

# Print the number of unique values in each column
print(unique_value_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [ ]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_type_value_counts = application_df['APPLICATION_TYPE'].value_counts()
print(app_type_value_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Identify values to replace with 'Other' (e.g., values that occur less frequently)
#Identify values to replace with 'Other' in "APPLICATION_TYPE" column
replace_values = app_type_value_counts[app_type_value_counts < 500].index

# Replace identified values with 'Other'
#Set inplace=True to have replacement done within application_df
application_df['APPLICATION_TYPE'].replace(replace_values, 'Other', inplace=True)

# Loop through dataframe to replace specific values that may not have been captured by .replace statement
for app in replace_values:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_value_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_value_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE
bottom_classification = classification_value_counts[(classification_value_counts > 1) & (classification_value_counts < 1000)]
print(bottom_classification)


CLASSIFICATION
C7000    777
C1700    287
C4000    194
C5000    116
C1270    114
C2700    104
C2800     95
C7100     75
C1300     58
C1280     50
C1230     36
C1400     34
C7200     32
C2300     32
C1240     30
C8000     20
C7120     18
C1500     16
C1800     15
C6000     15
C1250     14
C8200     11
C1238     10
C1278     10
C1235      9
C1237      9
C7210      7
C2400      6
C1720      6
C4100      6
C1257      5
C1600      5
C1260      3
C2710      3
C0         3
C3200      2
C1234      2
C1246      2
C1267      2
C1256      2
Name: count, dtype: int64


In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications_to_replace = classification_value_counts[classification_value_counts < 777].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [ ]:
column_list = application_df.columns
print(column_list)

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')


In [ ]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
#Field names: APPLICATION_TYPE	AFFILIATION	CLASSIFICATION	USE_CASE

application_type_value_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_value_counts)

affiliation_value_counts = application_df['AFFILIATION'].value_counts()
print(affiliation_value_counts)

use_case_value_counts = application_df['USE_CASE'].value_counts()
print(use_case_value_counts)

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64
AFFILIATION
Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: count, dtype: int64
USE_CASE
Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: count, dtype: int64


In [ ]:
#Field names: ORGANIZATION	STATUS INCOME_AMT

organization_value_counts = application_df['ORGANIZATION'].value_counts()
print(organization_value_counts)

status_value_counts = application_df['STATUS'].value_counts()
print(status_value_counts)

income_amount_value_counts = application_df['INCOME_AMT'].value_counts()
print(income_amount_value_counts)

ORGANIZATION
Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: count, dtype: int64
STATUS
1    34294
0        5
Name: count, dtype: int64
INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: count, dtype: int64


In [ ]:
#Field names:	SPECIAL_CONSIDERATIONS	IS_SUCCESSFUL
consideration_value_counts = application_df['SPECIAL_CONSIDERATIONS'].value_counts()
print(consideration_value_counts)

success_value_counts = application_df['IS_SUCCESSFUL'].value_counts()
print(success_value_counts)

SPECIAL_CONSIDERATIONS
N    34272
Y       27
Name: count, dtype: int64
IS_SUCCESSFUL
1    18261
0    16038
Name: count, dtype: int64


In [ ]:
# Make copy of original DataFrame
application_df_copy = application_df.copy()

#Field names: APPLICATION_TYPE	AFFILIATION	CLASSIFICATION	USE_CASE	ORGANIZATION INCOME_AMT	SPECIAL_CONSIDERATIONS
#Set'columns_to_encode' as list of columns containing categorical data
columns_to_encode = ['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT','SPECIAL_CONSIDERATIONS']  # Specify the columns to encode

# Apply pd.get_dummies to encode categorical columns
application_df_encoded = pd.get_dummies(application_df, columns=columns_to_encode)

# can customize column names of generated dummy variables
# For example, adding a prefix to the column names
application_df_encoded = pd.get_dummies(application_df, columns=columns_to_encode, prefix=columns_to_encode)

# Display DataFrame with encoded categorical data
print(application_df_encoded.head())

   STATUS  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0       1     5000              1                   False   
1       1   108590              1                   False   
2       1     5000              0                   False   
3       1     6692              1                   False   
4       1   142590              1                   False   

   APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                  True                 False                False   
1                 False                 False                 True   
2                 False                 False                False   
3                 False                 False                 True   
4                 False                 False                 True   

   APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  ...  \
0                False                False                False  ...   
1                False                False                False  

In [ ]:
# Skip this block. Already done above.
# Convert categorical data to numeric with `pd.get_dummies`
# Assuming 'df' is your DataFrame and 'column_to_encode' is the column containing categorical data
#column_to_encode = 'Category'  # Specify the column to encode

# Apply pd.get_dummies to encode the categorical column
#df_encoded = pd.get_dummies(application_df, columns=[column_to_encode])

# Optionally, you can customize the column names of the generated dummy variables
# For example, adding a prefix to the column names
#df_encoded = pd.get_dummies(df, columns=[column_to_encode], prefix=column_to_encode)

# Display the DataFrame with the encoded categorical data
#print(df_encoded.head())

In [ ]:
column_list = application_df_encoded.columns
print(column_list)

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_C7000', 'CLASSIFICATION_Other',
       'USE_CASE_CommunityServ', 'USE_CASE_Heathcare', 'USE_CASE_Other',
       'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_

In [ ]:
# Convert True and False to 1 and 0
# Specify columns to convert

columns_to_convert = ['APPLICATION_TYPE_Other', 'APPLICATION_TYPE_T10',
       'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3', 'APPLICATION_TYPE_T4',
       'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6', 'APPLICATION_TYPE_T7',
       'APPLICATION_TYPE_T8', 'AFFILIATION_CompanySponsored',
       'AFFILIATION_Family/Parent', 'AFFILIATION_Independent',
       'AFFILIATION_National', 'AFFILIATION_Other', 'AFFILIATION_Regional',
       'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200', 'CLASSIFICATION_C2000',
       'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000', 'CLASSIFICATION_C7000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
       'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M',
       'SPECIAL_CONSIDERATIONS_N', 'SPECIAL_CONSIDERATIONS_Y']

# Convert specified columns from boolean to integer
application_df_encoded[columns_to_convert] = application_df_encoded[columns_to_convert].astype(int)

print(application_df_encoded)

       STATUS   ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0           1      5000              1                       0   
1           1    108590              1                       0   
2           1      5000              0                       0   
3           1      6692              1                       0   
4           1    142590              1                       0   
...       ...       ...            ...                     ...   
34294       1      5000              0                       0   
34295       1      5000              0                       0   
34296       1      5000              0                       0   
34297       1      5000              1                       0   
34298       1  36500179              0                       0   

       APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                         1                     0                    0   
1                         0                     0          

In [ ]:
column_types = application_df_encoded.dtypes
print(column_types)

STATUS                          int64
ASK_AMT                         int64
IS_SUCCESSFUL                   int64
APPLICATION_TYPE_Other          int64
APPLICATION_TYPE_T10            int64
APPLICATION_TYPE_T19            int64
APPLICATION_TYPE_T3             int64
APPLICATION_TYPE_T4             int64
APPLICATION_TYPE_T5             int64
APPLICATION_TYPE_T6             int64
APPLICATION_TYPE_T7             int64
APPLICATION_TYPE_T8             int64
AFFILIATION_CompanySponsored    int64
AFFILIATION_Family/Parent       int64
AFFILIATION_Independent         int64
AFFILIATION_National            int64
AFFILIATION_Other               int64
AFFILIATION_Regional            int64
CLASSIFICATION_C1000            int64
CLASSIFICATION_C1200            int64
CLASSIFICATION_C2000            int64
CLASSIFICATION_C2100            int64
CLASSIFICATION_C3000            int64
CLASSIFICATION_C7000            int64
CLASSIFICATION_Other            int64
USE_CASE_CommunityServ          int64
USE_CASE_Hea

In [ ]:
# Split our preprocessed data into our features and target arrays
# Identify target variable
#target_variable = 'IS_SUCCESSFUL'

# Create target array (y)
#y = application_df_encoded[target_variable].values

# Create features array (X) by dropping target variable column
#X = application_df_encoded.drop(columns=[target_variable]).values

# Split the preprocessed data into a training and testing dataset
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [ ]:
#Scale only columns with continuous values

# Identify columns to scale
columns_to_scale = ['ASK_AMT']

# Extract values of columns to scale
data_to_scale = application_df_encoded[columns_to_scale].values

In [ ]:
#Scale extracted data, then replace original values with scaled values in DataFrame

# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform selected columns
scaled_data = scaler.fit_transform(data_to_scale)

# Replace original values with scaled values in application_df_encoded DataFrame
application_df_encoded[columns_to_scale] = scaled_data

In [ ]:
# Check data types of all columns in DataFrames
column_types = application_df_encoded.dtypes
print(column_types)

STATUS                            int64
ASK_AMT                         float64
IS_SUCCESSFUL                     int64
APPLICATION_TYPE_Other            int64
APPLICATION_TYPE_T10              int64
APPLICATION_TYPE_T19              int64
APPLICATION_TYPE_T3               int64
APPLICATION_TYPE_T4               int64
APPLICATION_TYPE_T5               int64
APPLICATION_TYPE_T6               int64
APPLICATION_TYPE_T7               int64
APPLICATION_TYPE_T8               int64
AFFILIATION_CompanySponsored      int64
AFFILIATION_Family/Parent         int64
AFFILIATION_Independent           int64
AFFILIATION_National              int64
AFFILIATION_Other                 int64
AFFILIATION_Regional              int64
CLASSIFICATION_C1000              int64
CLASSIFICATION_C1200              int64
CLASSIFICATION_C2000              int64
CLASSIFICATION_C2100              int64
CLASSIFICATION_C3000              int64
CLASSIFICATION_C7000              int64
CLASSIFICATION_Other              int64


In [ ]:
# Split preprocessed data into features and target arrays
# Identify target variable
target_variable = 'IS_SUCCESSFUL'

# Create target array (y)
y = application_df_encoded[target_variable].values

# Create features array (X) by dropping target variable column
X = application_df_encoded.drop(columns=[target_variable]).values

In [ ]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [ ]:
#Create scaler instance, fit scaler, scale data
# Colab keeps giving NameError: name 'sklearn' is not defined and 'skl' is not defined,
#so import again
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler

# Create scaler instance, fit scaler, scale data
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
#Compile, fit, and train model with callback to save model's weights every five epochs

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np

# Create Keras Sequential model
nn_model = tf.keras.models.Sequential()

# Define model
nn_model.add(tf.keras.layers.Dense(units=6, activation="relu", input_dim=44))
nn_model.add(tf.keras.layers.Dense(units=6, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check structure of Sequential model
nn_model.summary()

# Compile model with accuracy metric
nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define callback to save model's weights every five epochs
checkpoint_path = "model_weights.h5"
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_freq=5 * len(X_train),)  # save every 5 epochs (converted to number of samples)

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3)

callbacks_list = [checkpoint_callback, early_stopping_callback]

# Train model
history = nn_model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_test, y_test),
    callbacks=callbacks_list)

# Access and print details from callbacks after training
print("Model weights saved at:", checkpoint_callback.filepath)
print("Early stopping was triggered:", early_stopping_callback.stopped_epoch)

# Access callbacks from model history (note: not best practice)
# demonstrates that callbacks are referenced in model's history
print("Early stopping was triggered at epoch:", early_stopping_callback.stopped_epoch)


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 6)                 270       
                                                                 
 dense_13 (Dense)            (None, 6)                 42        
                                                                 
 dense_14 (Dense)            (None, 1)                 7         
                                                                 
Total params: 319 (1.25 KB)
Trainable params: 319 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
804/804 [==============================] - 4s 3ms/step - loss: 0.6147 - accuracy: 0.6865 - val_loss: 0.5856 - val_accuracy: 0.7142
Epoch 2/10
804/804 [==============================] - 3s 3ms/step - loss: 0.5727 - accuracy: 0.7249 - val_loss: 0.5713 - val_accuracy: 0.7201
Epoc

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7340 - accuracy: 0.5569 - 390ms/epoch - 1ms/step
Loss: 0.7339599132537842, Accuracy: 0.556851327419281


In [ ]:
# Export our model to HDF5 file
import h5py

# Specify filename
filename = "results.h5"

# Extract values from history object
history_dict = {
    'loss': history.history['loss'],
    'accuracy': history.history['accuracy'],
    'val_loss': history.history['val_loss'],
    'val_accuracy': history.history['val_accuracy']}

# Create new HDF5 file
with h5py.File(filename, 'w') as f:
    # Save each dataset in the history dictionary
    for key, value in history_dict.items():
        f.create_dataset(key, data=value)